# CHPT 4 - Sur-ajustement et sous-ajustement : identification et résolution

- outre l'utilisation de réseaux de neurones de capacité différentes (nombre de couche, taille des couches) deux méthodes sont couramment employées pour éviter le sur-ajustement
- l'ajout de régularisation sur les coefficients des couches (L1 ou L2) plus souvent L2
- l'ajout de dropout après les couches permettant d'éteindre aléatoirement des neuronesdans la couche précédentes (souvent entre 20 et 50 %)

## Régularisation norme l1 et l2

In [13]:
from keras.datasets import imdb
import numpy as np

(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words = 10000)

def vectorize_data(sequences,dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence]=1
    return results

x_train = vectorize_data(train_data)
y_train = np.asarray(train_labels).astype(float)

x_test = vectorize_data(test_data)
y_test = np.asarray(test_labels).astype(float)

In [14]:
from keras import layers, models, regularizers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(16, activation = "relu",input_shape = (10000,)))
    model.add(layers.Dense(16, activation = "relu"))
    model.add(layers.Dense(1,activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    return model

def build_model_l2():
    model = models.Sequential()
    model.add(layers.Dense(16, activation = "relu",kernel_regularizer = regularizers.l2(0.001),input_shape = (10000,)))
    model.add(layers.Dense(16, activation = "relu",kernel_regularizer = regularizers.l2(0.001)))
    model.add(layers.Dense(1,activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    return model

model_l2 = build_model_l2()
model_l2.fit(x_train,y_train,epochs=5,batch_size = 512)

Epoch 1/5
49/49 [==============================] - 1s 16ms/step - loss: 0.4925 - accuracy: 0.8250
Epoch 2/5
49/49 [==============================] - 1s 16ms/step - loss: 0.3150 - accuracy: 0.9061 0s - loss: 0.3157 - accuracy: 0.90
Epoch 3/5
49/49 [==============================] - 1s 17ms/step - loss: 0.2702 - accuracy: 0.9215
Epoch 4/5
49/49 [==============================] - 1s 20ms/step - loss: 0.2520 - accuracy: 0.9288
Epoch 5/5
49/49 [==============================] - 1s 18ms/step - loss: 0.2421 - accuracy: 0.9321


In [9]:
model.evaluate(x_test,y_test)


782/782 [==============================] - 1s 2ms/step - loss: 0.3426 - accuracy: 0.8838


[0.34258249402046204, 0.883840024471283]